In [87]:
import yfinance as yf
import pandas as pd
import numpy as np


tickers = [
    'TUPRS.IS', 
    'TCELL.IS', 
    'CCOLA.IS', 
    'SISE.IS', 
    'ENJSA.IS', 
    'ARCLK.IS', 
    'OTKAR.IS', 
    'SELEC.IS',
    'AKSA.IS',
    'DGNMO.IS'
]

start_date = '2016-01-01'  # YYYY-MM-DD
end_date = '2025-10-03'
interval = '1d'
filename = 'Turkish_10_companies.xlsx'


def calculate(tickers, start, end, interval):
    df = yf.download(tickers, start=start, end=end, interval=interval, auto_adjust=True)
    # when we first argument is list we download data for several stocks
    # as a result we get a table with multi-level columns, where level=0 (higher order) columns are Close, High, Low, Open, Volume
    # level=1 columns are tickers
    
    # Claculate daily returns for each stock as percentage change and append it original df to keep multi-column structure
    daily_returns = df['Close'].pct_change()
    daily_returns.columns = pd.MultiIndex.from_product([['Daily Return'], daily_returns.columns]) # create multi-column structure similar to df one
    df = pd.concat([df, daily_returns], axis=1) # label-alignment and appending from the right (label-alignment)

    # calculating yearly return. To avoid possible missing prices we check if a stock has prices at all at particular year (drop na values)
    # and in this shrinked series take first and last values available
    # first apply considers subdf as variable, second apply considers separate ticker series as variable .dropna() shrinks series to valid
    # number .size checks if this shrinked series has values at all

    year = df.index.year
    first_prices = df['Close'].groupby(year).apply(lambda subdf: subdf.apply(lambda ticker_series: ticker_series.dropna().iloc[0] if ticker_series.dropna().size else np.nan))
    last_prices = df['Close'].groupby(year).apply(lambda subdf: subdf.apply(lambda ticker_series: ticker_series.dropna().iloc[-1] if ticker_series.dropna().size else np.nan))
    return_yearly = last_prices/first_prices-1 # label and column alignment happens here
    # making multicolumn structure
    return_yearly.columns = pd.MultiIndex.from_product([['Yearly return'], return_yearly.columns])

    # now we calculate average yearly return and daily volatility of returns for each year

    daily_ret = df['Close'].pct_change()
    avg_daily = daily_ret.groupby(year).mean()
    avg_daily.columns = pd.MultiIndex.from_product([['Average daily return'], avg_daily.columns])
    vol_daily = daily_ret.groupby(year).std(ddof=1)
    vol_daily.columns = pd.MultiIndex.from_product([['Daily volatility'], vol_daily.columns])
    
    # we calculate yearly volatility as daily standard deviation of returns multiplied by square root of N:
    # intuition: daily returns r(t) are assumed to be independent and indentically distributed with same mean and standard deviation. Hence 
    # Var(sum(r(t)) = Var(r(1)) + ...Var(r(t)) = N * (std.dev)^2. Hence yearly volatility is compute as sqrt(number of days in a year) * std.dev
    
    days_in_years = daily_ret.groupby(year).count()
    vol_yearly = vol_daily * np.sqrt(days_in_years)
    vol_yearly.rename(columns={'Daily volatility': 'Yearly volatility'}, inplace=True)
    df2 = pd.concat([return_yearly, avg_daily, vol_daily, vol_yearly], axis=1)
    print(df2.columns)
    
    with pd.ExcelWriter(filename) as writer:
        df.to_excel(writer, sheet_name='prices', na_rep='N/A')
        df2.to_excel(writer, sheet_name='calculated_indicators', na_rep='N/A')

result_df = calculate(tickers, start_date, end_date, interval)

[*********************100%***********************]  10 of 10 completed
/var/folders/08/ph8clkwj3jg7b4ky_zw3ztwc0000gn/T/ipykernel_4035/4240841871.py:32: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily_returns = df['Close'].pct_change()
/var/folders/08/ph8clkwj3jg7b4ky_zw3ztwc0000gn/T/ipykernel_4035/4240841871.py:50: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily_ret = df['Close'].pct_change()


MultiIndex([(       'Yearly return',  'AKSA.IS'),
            (       'Yearly return', 'ARCLK.IS'),
            (       'Yearly return', 'CCOLA.IS'),
            (       'Yearly return', 'DGNMO.IS'),
            (       'Yearly return', 'ENJSA.IS'),
            (       'Yearly return', 'OTKAR.IS'),
            (       'Yearly return', 'SELEC.IS'),
            (       'Yearly return',  'SISE.IS'),
            (       'Yearly return', 'TCELL.IS'),
            (       'Yearly return', 'TUPRS.IS'),
            ('Average daily return',  'AKSA.IS'),
            ('Average daily return', 'ARCLK.IS'),
            ('Average daily return', 'CCOLA.IS'),
            ('Average daily return', 'DGNMO.IS'),
            ('Average daily return', 'ENJSA.IS'),
            ('Average daily return', 'OTKAR.IS'),
            ('Average daily return', 'SELEC.IS'),
            ('Average daily return',  'SISE.IS'),
            ('Average daily return', 'TCELL.IS'),
            ('Average daily return', 'TUPRS.IS'),


In [43]:
# here we c



Price            Close                   High                    Low  \
Ticker            AAPL       GOOG        AAPL       GOOG        AAPL   
Date                                                                   
2022-11-01  148.308868  89.884895  153.034279  95.511393  146.812500   
2022-11-02  142.776199  86.478203  149.805242  90.679456  142.746667   
2022-11-03  136.721817  82.922539  140.580899  85.961746  136.593832   
2022-11-04  136.455582  86.110710  140.685915  86.140513  132.511209   
2022-11-07  136.988022  88.047478  137.214819  88.335508  133.783220   
2022-11-08  137.559937  88.305702  139.463088  89.790536  135.577896   
2022-11-09  132.994339  86.805969  136.623169  88.881760  132.718234   
2022-11-10  144.827469  93.529945  144.827469  93.907367  137.559969   
2022-11-11  147.618103  96.072556  147.923789  96.698271  142.362226   
2022-11-14  146.217865  95.377304  148.190051  96.519489  145.379680   

Price                        Open                Volume            
Ticker           GOOG        AAPL       GOOG       AAPL      GOOG  
Date                                                               
2022-11-01  89.815371  152.670033  94.940296   80379300  43220600  
2022-11-02  86.418613  146.635280  90.292107   93604600  43553600  
2022-11-03  82.882810  139.852393  85.758137   97918500  48510400  
2022-11-04  83.309878  140.113979  84.928804  140814800  40173300  
2022-11-07  86.368962  135.203196  86.746376   83374600  26899900  
2022-11-08  87.054264  138.457285  88.554003   89908500  30172000  
2022-11-09  86.766240  136.573861  87.943184   74917800  26743900  
2022-11-10  91.027077  139.275776  91.712382  118854000  42371200  
2022-11-11  93.520023  143.792073  94.066281   93979700  30569100  
2022-11-14  94.466537  146.898272  94.850908   73374100  24170100